In [10]:
import os
import re

import pandas

In [11]:
# entropy = pandas.read_json('data/output/entropy/sample_0.jsonl', lines=True)
# entropy.head(1)

In [12]:
# joined = pandas.read_json('data/output/joined/sample_0.jsonl', lines=True)
# joined['is_correct'] = joined['A. ChatGPT gets it correct from text to final solution']
# joined['is_wrong'] = 1 - joined['is_correct']
# joined = joined[['question_id', 'is_wrong']]
# joined.head(1)

In [13]:
# combined = joined.set_index('question_id').join(entropy.set_index('question_id'))
# combined

In [14]:
# def generate_geq(row, column, step_size):
#     for i in range(0, 50, step_size):
#         i = round(i * 0.1, 1)
#         if row[column] > i: row[f'{column}_geq_{i}'] = True
#         else: row[f'{column}_geq_{i}'] = False
#     return row

In [15]:
def do_geq(column, combined):
    to_graph = []
    for i in range(0, 50, 1):
        i = i * 0.1
        one = combined[combined[column] >= i]
        if len(one) == 0: break
        to_graph.append({
            'geq_value': i, 
            'number of incorrect': one['is_wrong'].sum(), 
            'set size': len(one), 
            'probability_of_failure': one['is_wrong'].sum() / len(one)
        })

    to_graph = pandas.DataFrame(to_graph)
    return to_graph

def do_leq(column, combined):
    to_graph = []
    for i in range(0, 50, 1):
        i = i * 0.1
        one = combined[combined[column] <= i]
        to_graph.append({
            'leq_value': i, 
            'number of incorrect': one['is_wrong'].sum(), 
            'set size': len(one), 
            'probability_of_failure': one['is_wrong'].sum() / len(one)
        })
        if len(one) == len(combined): break

    to_graph = pandas.DataFrame(to_graph)
    return to_graph

In [16]:
def do_geq_file(file):
    entropy = pandas.read_json(file, lines=True)
    entropy.head(1)
    joined = pandas.read_json('data/output/joined/sample_0.jsonl', lines=True)
    joined['is_correct'] = joined['A. ChatGPT gets it correct from text to final solution']
    joined['is_wrong'] = 1 - joined['is_correct']
    joined = joined[['question_id', 'is_wrong']]
    joined.head(1)
    combined = joined.set_index('question_id').join(entropy.set_index('question_id'))
    
    file_name = file.split('/')[-2]
    if not os.path.exists(f'data/output/xlsx/Entropy-Failure (GEQ)/'):
        os.makedirs(f'data/output/xlsx/Entropy-Failure (GEQ)/')

    writer = pandas.ExcelWriter(f'data/output/xlsx/Entropy-Failure (GEQ)/{file_name}.xlsx', engine='xlsxwriter')
    workbook = writer.book

    for i in [
        {'name': 'Answer', 'column': 'entropy_answer'},
        {'name': 'Strict', 'column': 'entropy_equation_strict'},
        {'name': 'Loose', 'column': 'entropy_equation_loose'}
    ]:
        name = i['name']
        column = i['column']
        
        wrap_format = workbook.add_format()
        wrap_format.set_text_wrap()

        answer = do_geq(column, combined)
        answer.to_excel(writer, sheet_name=name)
        chart1 = workbook.add_chart({"type": "scatter", "subtype": "straight_with_markers"})
        chart1.add_series(
        {
            "name": name,
            "categories": f"={name}!$B$2:$B$50",
            "values": f"={name}!$E$2:$E$50",
        })
        chart1.set_y_axis({'min': 0, 'max': 1})
                
        chart1.set_legend({'position': 'overlay_right'})
        chart1.set_style(10)
        chart1.set_size({'width': 1000, 'height': 600})

        sheet = writer.sheets[name]
        sheet.insert_chart('H1', chart1, {'x_offset': 0, 'y_offset': 0})

        sheet.set_column('A:H', 20, wrap_format)
    writer.close()

In [17]:
def do_leq_file(file):
    entropy = pandas.read_json(file, lines=True)
    entropy.head(1)
    joined = pandas.read_json('data/output/joined/sample_0.jsonl', lines=True)
    joined['is_correct'] = joined['A. ChatGPT gets it correct from text to final solution']
    joined['is_wrong'] = 1 - joined['is_correct']
    joined = joined[['question_id', 'is_wrong']]
    joined.head(1)
    combined = joined.set_index('question_id').join(entropy.set_index('question_id'))
    
    file_name = file.split('/')[-2]
    if not os.path.exists(f'data/output/xlsx/Entropy-Failure (LEQ)/'):
        os.makedirs(f'data/output/xlsx/Entropy-Failure (LEQ)/')

    writer = pandas.ExcelWriter(f'data/output/xlsx/Entropy-Failure (LEQ)/{file_name}.xlsx', engine='xlsxwriter')
    workbook = writer.book

    for i in [
        {'name': 'Answer', 'column': 'entropy_answer'},
        {'name': 'Strict', 'column': 'entropy_equation_strict'},
        {'name': 'Loose', 'column': 'entropy_equation_loose'}
    ]:
        name = i['name']
        column = i['column']
        
        wrap_format = workbook.add_format()
        wrap_format.set_text_wrap()

        answer = do_leq(column, combined)
        answer.to_excel(writer, sheet_name=name)    
        chart1 = workbook.add_chart({"type": "scatter", "subtype": "straight_with_markers"})
        chart1.add_series(
        {
            "name": name,
            "categories": f"={name}!$B$2:$B$50",
            "values": f"={name}!$E$2:$E$50",
        })
        chart1.set_y_axis({'min': 0, 'max': 1})
                
        chart1.set_legend({'position': 'overlay_right'})
        chart1.set_style(10)
        chart1.set_size({'width': 1000, 'height': 600})
        chart1.set_title({'name': name})

        sheet = writer.sheets[name]
        sheet.insert_chart('H1', chart1, {'x_offset': 0, 'y_offset': 0})

        sheet.set_column('A:H', 20, wrap_format)
    writer.close()

In [18]:
do_geq_file('data/output/Entropy/sample_0.jsonl')
do_geq_file('data/output/Entropy Sympy != Ans/sample_0.jsonl')
do_geq_file('data/output/Entropy Sympy = Ans/sample_0.jsonl')

do_leq_file('data/output/Entropy/sample_0.jsonl')
do_leq_file('data/output/Entropy Sympy != Ans/sample_0.jsonl')
do_leq_file('data/output/Entropy Sympy = Ans/sample_0.jsonl')